In [2]:
import torch
import torchaudio
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
import pandas as pd
import torchaudio
import torchaudio.transforms as T
import torch
import csv

/home/jdecim/Documents/EPFL/Master/MA2/Hackathon/Lemanic-Life-Sciences-Hackathon-2025/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from transformers import AutoTokenizer, AutoModel
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
from text2phonemesequence import Text2PhonemeSequence

text2phone_model = Text2PhonemeSequence(language='jpn', is_cuda=True)
sentence = "これ は 、 テスト テキスト です ."
input_phonemes = text2phone_model.infer_sentence(sentence)


pytorch_model.bin:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

In [1]:
import torch
import torchaudio
from torchaudio.pipelines import WAV2VEC2_ASR_BASE_960H

# Load Wav2Vec2 model and bundle
bundle = WAV2VEC2_ASR_BASE_960H
model = bundle.get_model()

# Load and preprocess audio
def load_audio(path, target_sr=16000):
    waveform, sr = torchaudio.load(path)
    if sr != target_sr:
        waveform = torchaudio.functional.resample(waveform, sr, target_sr)
    return waveform.squeeze(0), target_sr

# Get emissions (acoustic feature probabilities from Wav2Vec2)
def get_emissions(model, waveform):
    with torch.inference_mode():
        emissions, _ = model(waveform.unsqueeze(0))  # shape: (1, time, classes)
    return torch.log_softmax(emissions[0], dim=-1)

# Force alignment using torchaudio's CTC decoder and alignment
def forced_align(emissions, transcript, tokens, sample_rate, blank=0):
    from torchaudio.functional import forced_align
    # Convert transcript to indices
    targets = torch.tensor([tokens[c] for c in transcript if c in tokens], dtype=torch.int32)
    alignment = forced_align(emissions, targets, blank=blank)
    time_per_frame = emissions.size(0) / sample_rate
    results = []
    for i, (t, duration) in enumerate(alignment):
        phoneme = list(tokens.keys())[list(tokens.values()).index(t)]
        start = i * time_per_frame
        end = start + duration * time_per_frame
        results.append((phoneme, start, end))
    return results

# Define phoneme tokens manually (simple mock)
phoneme_tokens = {c: i + 1 for i, c in enumerate("aeiouʃkstnmrl|")}  # Example set
phoneme_tokens["_"] = 0  # CTC blank

# Your test audio
audio_path = "your_audio.wav"
waveform, sr = load_audio(audio_path)
emissions = get_emissions(model, waveform)

# Provide a phoneme transcript manually (or from another model)
phoneme_transcript = "ʃi|pɛr|sɔna"  # Replace this with actual transcription if available

# Get timestamps
phoneme_timestamps = forced_align(emissions, phoneme_transcript, phoneme_tokens, sample_rate=sr)

# Output
for ph, start, end in phoneme_timestamps:
    print(f"{ph}: {start:.2f}s - {end:.2f}s")


/home/jdecim/Documents/EPFL/Master/MA2/Hackathon/Lemanic-Life-Sciences-Hackathon-2025/.venv/lib/python3.11/site-packages/torch/nn/modules/transformer.py:20: UserWarning: Failed to initialize NumPy: No module named 'numpy' (Triggered internally at ../torch/csrc/utils/tensor_numpy.cpp:84.)
  device: torch.device = torch.device(torch._C._get_default_device()),  # torch.device('cpu'),


RuntimeError: Failed to open the input "your_audio.wav" (No such file or directory).
Exception raised from get_input_format_context at /__w/audio/audio/pytorch/audio/src/libtorio/ffmpeg/stream_reader/stream_reader.cpp:42 (most recent call first):
frame #0: c10::Error::Error(c10::SourceLocation, std::string) + 0x57 (0x714fc7681d87 in /home/jdecim/Documents/EPFL/Master/MA2/Hackathon/Lemanic-Life-Sciences-Hackathon-2025/.venv/lib/python3.11/site-packages/torch/lib/libc10.so)
frame #1: c10::detail::torchCheckFail(char const*, char const*, unsigned int, std::string const&) + 0x64 (0x714fc763275f in /home/jdecim/Documents/EPFL/Master/MA2/Hackathon/Lemanic-Life-Sciences-Hackathon-2025/.venv/lib/python3.11/site-packages/torch/lib/libc10.so)
frame #2: <unknown function> + 0x42904 (0x714fc8f64904 in /home/jdecim/Documents/EPFL/Master/MA2/Hackathon/Lemanic-Life-Sciences-Hackathon-2025/.venv/lib/python3.11/site-packages/torio/lib/libtorio_ffmpeg6.so)
frame #3: torio::io::StreamingMediaDecoder::Str

In [86]:
phoneme_seq

["<unk>NETRNLUENLEXVSH<unk>IS'KESUZ<unk>S'NE</s>ELDOBTRESB<s>UATHTDWHWX'HWHWCNAWAH'WXWEB</s>E'UCBCEZBC'E</s>CA'</s>E'U'</s>UHY'NSES'YNYISESESINVESEIEUNS</s>SUSUS</s>",
 "U<s>U'R</s>U</s>UY'<unk></s>'SESKN</s>N</s>ZNEB<unk>KHYHUWYW'YS'YKXD</s>S</s>XKX'SXS</s>STXPXU</s>'UYKES'P'X'S'NEIQ</s>'X'S'YHYA'<unk>'</s>KE</s>V</s>UXNEVNULS<unk>Q<unk>XEL'<unk>NEZT</s>LELES<unk>X<unk>AW</s>U</s>UC</s>CK</s>V</s>EL<unk>Z</s><s>HKE",
 "GLEDEKNXKE'IDHYH<unk>YUY<unk>Y</s>EUY'YHY'</s>ERTE</s>EH</s>H'ABT'K</s>XH</s>D</s>SD<s>HWE'EB'BUB</s>U'U'SESUS</s>T</s>EKE</s>EUQ'RECEUCQ</s>Q</s>ELC",
 'BO</s>KD',
 'D',
 "EUEUEK<unk>KESDWYH<unk>HYH'YAY<unk>YWZE</s>E</s>S'SX<unk>KENEUS<unk>E<unk>'<unk>'<unk>YSUT<unk>'SX<unk>ESXS'<unk>SXTYKSESESXTQKEIS</s>M<unk></s>EUT'</s>ES'<unk></s>X'S<unk>XESEK",
 "EZDOQOXEZE<unk><s>QTEUKESKSMHTDHD</s>HD</s>HDYDTDEDH'WHTE</s>KHKDKH'<unk></s>E'C</s>X'R'YUY'Y'YS'HY'</s>E</s>US</s>REUEUEZEDIDAKECE<unk><s>NXKSES<s>XASXRURESREU'<s>'RXR'YSRU'REURQ'Q'R'EQ<s></s>'SDPRBR'ES'ES'R'U'R'RURK'RU'

In [ ]:
# Load model with forced alignment support
bundle = torchaudio.pipelines.WAVLM_BASE_PLUS
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model = bundle.get_model().to(device)
labels = bundle.get_labels()

# Load and preprocess audio
def load_audio(audio_path):
    waveform, sample_rate = torchaudio.load(audio_path)
    if sample_rate != bundle.sample_rate:
        resampler = T.Resample(orig_freq=sample_rate, new_freq=bundle.sample_rate)
        waveform = resampler(waveform)
    return waveform.squeeze(0)

# Greedy decode with time info
def forced_alignment(waveform):
    with torch.inference_mode():
        emissions, _ = model(waveform[None])
    emissions = torch.log_softmax(emissions, dim=-1)

    # Greedy decoding with timestamps
    tokens = torch.argmax(emissions, dim=-1).squeeze(0)
    timestamps = []

    prev = None
    start_frame = 0
    time_per_frame = waveform.shape[0] / emissions.shape[1] / bundle.sample_rate

    for i, token in enumerate(tokens):
        if token != prev:
            if prev is not None and prev != bundle.get_labels().index("|"):  # Skip blanks
                label = labels[prev]
                start = start_frame * time_per_frame
                end = i * time_per_frame
                timestamps.append((label, round(start, 2), round(end, 2)))
            start_frame = i
            prev = token

    # Add last token
    if prev is not None and prev != bundle.get_labels().index("|"):
        label = labels[prev]
        start = start_frame * time_per_frame
        end = len(tokens) * time_per_frame
        timestamps.append((label, round(start, 2), round(end, 2)))

    return timestamps

# Run
audio_path_folder = "data/2_Audiofiles/Decoding_IT_T1/"
audio_file = "102_edugame2023_32c4a5e851c1431aba3aa409e3be8128_649d404f44214261b67b24f1845e1350.wav"
audio_path = audio_path_folder + audio_file
waveform = load_audio(audio_path)
phoneme_timestamps = forced_alignment(waveform)
phonemes=[]
timestamps = []
for ph, start, end in phoneme_timestamps:
    phonemes.append(ph)
    timestamps.append([start, end])
    
csv_file = "phoneme_timestamps.csv"

# Write header if creating a new file
write_header = True  # Change to False if you're appending more files later

with open(csv_file, mode='w' if write_header else 'a', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    if write_header:
        writer.writerow(["audio_file", "phonemes", "timestamps"])
    writer.writerow([audio_file, str(phonemes), str(timestamps)])

cpu
-: 0.00s - 0.02s
C: 0.02s - 0.04s
-: 0.04s - 0.08s
A: 0.08s - 0.10s
-: 0.10s - 0.12s
N: 0.12s - 0.14s
-: 0.14s - 0.20s
-: 0.22s - 0.38s
K: 0.38s - 0.40s
-: 0.40s - 0.46s
I: 0.46s - 0.48s
-: 0.48s - 0.54s
M: 0.56s - 0.60s
-: 0.60s - 0.64s
A: 0.64s - 0.66s
-: 0.66s - 0.74s
P: 0.74s - 0.76s
-: 0.76s - 0.86s
A: 0.86s - 0.88s
-: 0.88s - 1.00s
-: 1.04s - 1.08s
J: 1.08s - 1.10s
-: 1.10s - 1.16s
O: 1.16s - 1.18s
U: 1.18s - 1.20s
-: 1.22s - 1.26s
N: 1.26s - 1.28s
-: 1.28s - 1.32s
O: 1.32s - 1.34s
W: 1.34s - 1.36s
-: 1.36s - 1.42s
-: 1.46s - 5.44s
O: 5.44s - 5.46s
-: 5.46s - 5.56s
S: 5.56s - 5.58s
-: 5.58s - 5.78s
O: 5.78s - 5.80s
-: 5.80s - 5.90s
-: 5.92s - 8.48s
M: 8.48s - 8.50s
-: 8.50s - 8.54s
I: 8.54s - 8.56s
-: 8.56s - 8.64s
S: 8.64s - 8.70s
-: 8.70s - 8.74s
-: 8.76s - 8.80s
M: 8.80s - 8.82s
-: 8.82s - 8.94s
A: 8.94s - 8.96s
-: 8.96s - 8.98s
R: 8.98s - 9.00s
-: 9.00s - 9.12s
-: 9.14s - 9.20s
G: 9.20s - 9.22s
-: 9.22s - 9.34s
O: 9.34s - 9.36s
-: 9.36s - 9.44s
-: 9.48s - 9.90s
S: 9.90s -

In [22]:
timestamps

[[0.0, 0.02],
 [0.02, 0.04],
 [0.04, 0.08],
 [0.08, 0.1],
 [0.1, 0.12],
 [0.12, 0.14],
 [0.14, 0.2],
 [0.22, 0.38],
 [0.38, 0.4],
 [0.4, 0.46],
 [0.46, 0.48],
 [0.48, 0.54],
 [0.56, 0.6],
 [0.6, 0.64],
 [0.64, 0.66],
 [0.66, 0.74],
 [0.74, 0.76],
 [0.76, 0.86],
 [0.86, 0.88],
 [0.88, 1.0],
 [1.04, 1.08],
 [1.08, 1.1],
 [1.1, 1.16],
 [1.16, 1.18],
 [1.18, 1.2],
 [1.22, 1.26],
 [1.26, 1.28],
 [1.28, 1.32],
 [1.32, 1.34],
 [1.34, 1.36],
 [1.36, 1.42],
 [1.46, 5.44],
 [5.44, 5.46],
 [5.46, 5.56],
 [5.56, 5.58],
 [5.58, 5.78],
 [5.78, 5.8],
 [5.8, 5.9],
 [5.92, 8.48],
 [8.48, 8.5],
 [8.5, 8.54],
 [8.54, 8.56],
 [8.56, 8.64],
 [8.64, 8.7],
 [8.7, 8.74],
 [8.76, 8.8],
 [8.8, 8.82],
 [8.82, 8.94],
 [8.94, 8.96],
 [8.96, 8.98],
 [8.98, 9.0],
 [9.0, 9.12],
 [9.14, 9.2],
 [9.2, 9.22],
 [9.22, 9.34],
 [9.34, 9.36],
 [9.36, 9.44],
 [9.48, 9.9],
 [9.9, 9.92],
 [9.92, 10.28],
 [10.32, 10.98],
 [10.98, 11.0],
 [11.0, 11.1],
 [11.1, 11.12],
 [11.12, 11.26],
 [11.28, 11.32],
 [11.32, 11.34],
 [11.34, 11

In [ ]:

audio_file = "102_edugame2023_32c4a5e851c1431aba3aa409e3be8128_649d404f44214261b67b24f1845e1350.wav"
# Output CSV file


In [39]:
# # replace space with 
# import re
# df_cod1 = pd.read_csv("data/processed/Decoding_ground_truth_IT.csv",index_col=0)
# def space_phonemes(text):
#     # First, temporarily replace special tokens with placeholders
#     text = text.replace('[PAD]', '<<PAD>>').replace('[UNK]', '<<UNK>>')
#     # Now insert space between every character
#     text = ' '.join(text)
#     # Restore the special tokens
#     text = text.replace('< < P A D > >', '[PAD]').replace('< < U N K > >', '[UNK]')
#     # Clean up accidental spacing inside tokens (just in case)
#     text = re.sub(r'\[\s*PAD\s*\]', '[PAD]', text)
#     text = re.sub(r'\[\s*UNK\s*\]', '[UNK]', text)
#     return text

# # Replace space with | first
# df_cod1['trial_answer_coder2_phoneme'] = df_cod1['trial_answer_coder2_phoneme'].str.replace(' ', '|')

# # Then apply spacing logic
# df_cod1['trial_answer_coder2_phoneme'] = df_cod1['trial_answer_coder2_phoneme'].apply(space_phonemes)


In [53]:
df_cod1.iloc[-1][-1]

/tmp/ipykernel_934485/2641897723.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df_cod1.iloc[-1][-1]


'v a l o [PAD] | s a v e [PAD] | [UNK] [PAD] | k l o [PAD] | f e [PAD] | l o [PAD] | [UNK] [PAD] | s o n [PAD] | v i [PAD] | ʎ ɔ [PAD] | d e t e [PAD] | f a r d e [PAD] | p o r i t a [PAD] | p o r i t ː a [PAD] | p r o d i ʎ a [PAD] | a n [PAD] | t ʃ i [PAD] | b i ɲ a z o [PAD] | f l a n ɛ s t r o [PAD] | k o [PAD] | ɲ a [PAD] | r ɪ [PAD] | p o'

In [22]:
import pandas as pd
df_cod1 = pd.read_csv("data/processed/2_Decoding_ground_truth_IT.csv",index_col=0)
df_cod1['trial_answer_coder2_phoneme'][0]

'vuzo[PAD]seɡa[PAD]klofɛno[PAD]raviʎo[PAD]da[PAD]pe[PAD]tarse[PAD]doridzːa[PAD]prateʎa[PAD]aː[PAD]ɛrɾe[PAD]lo[PAD]beɲole[PAD]fla[PAD]vɛstro[PAD]kʊɲaripːo'

In [24]:
import re
import json
import os
def extract_unique_phonemes(phoneme_str):
    pattern = r"\[PAD\]|\[UNK\]|."
    phonemes = re.findall(pattern, phoneme_str)
    return list(dict.fromkeys(phonemes))
phonoemes = df_cod1['trial_answer_coder2_phoneme'].tolist()
phonoemes = "".join(phonoemes)
unique_phonemes_it = extract_unique_phonemes(phonoemes)
unique_phonemes_dict_it = {i: ph for i, ph in enumerate(unique_phonemes_it)}


with open("data/processed/unique_phonemes_IT.json", "w", encoding="utf-8") as f:
    json.dump(unique_phonemes_dict_it, f, ensure_ascii=False, indent=2)


In [25]:
len(phonoemes)

201409

In [26]:
len(unique_phonemes_it)

35

In [27]:
unique_phonemes_it

['v',
 'u',
 'z',
 'o',
 '[PAD]',
 's',
 'e',
 'ɡ',
 'a',
 'k',
 'l',
 'f',
 'ɛ',
 'n',
 'r',
 'i',
 'ʎ',
 'd',
 'p',
 't',
 'ː',
 'ɾ',
 'b',
 'ɲ',
 'ʊ',
 'm',
 'ɔ',
 'ʒ',
 'ɪ',
 'ʃ',
 '[UNK]',
 'w',
 'ŋ',
 'j',
 '̪']

In [28]:
df_cod_fr=pd.read_csv("data/processed/2_Phoneme_Deleletion_ground_truth_FR.csv",index_col=0)
df_cod_fr.dropna(inplace=True)

In [30]:
def extract_unique_phonemes(phoneme_str):
    pattern = r"\[PAD\]|\[UNK\]|."
    phonemes = re.findall(pattern, phoneme_str) 
    # remove ' ' this should not be in the list
    phonemes = [ph for ph in phonemes if ph != ' ']
    return list(dict.fromkeys(phonemes))
phonoemes = df_cod_fr['trial_answer_coder1_phoneme'].dropna().tolist()
phonoemes = "".join(phonoemes)
unique_phonemes_fr = extract_unique_phonemes(phonoemes)
unique_phonemes_dict_fr = {i: ph for i, ph in enumerate(unique_phonemes_fr)}

In [31]:
with open("data/processed/unique_phonemes_FR.json", "w", encoding="utf-8") as f:
    json.dump(unique_phonemes_dict_fr, f, ensure_ascii=False, indent=2)

In [32]:
# find unique phonemes in the list between two languages
unique_phonemes_fr = set(unique_phonemes_fr)
unique_phonemes_it = set(unique_phonemes_it)
unique_phonemes_fr_it = unique_phonemes_fr.union(unique_phonemes_it)
unique_phonemes_fr_it = list(unique_phonemes_fr_it)
unique_phonemes_fr_it = {i: ph for i, ph in enumerate(unique_phonemes_fr_it)}
with open("data/processed/unique_phonemes_FR_IT.json", "w", encoding="utf-8") as f:
    json.dump(unique_phonemes_fr_it, f, ensure_ascii=False, indent=2)

In [10]:
import pandas as pd
df=pd.read_csv('output_IT.csv')
df['probabilities'] = df['probabilities'].apply(eval)
df['timestamps'] = df['timestamps'].apply(eval)
len(df['probabilities'][0][0])

51

In [ ]:
df